<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Transform Secrets Engine
Transform is a Secret Engine that allows Vault to encode and decode sensitive values residing in external systems such as databases or filesystems.
Transform is capable of two types of transformations:
* masking - one way transformation that allows for Vault to anonymize data
* data type protection - another mode to encode and decode sensitive data while protecting the structure of data
<img src=images/vault-transform-1.png width="800">

**Note:** This secret engine requires Vault Enterprise with Advanced Data Protection module.

The Transform secrets engine handles secure data transformation and tokenization against input value.  Transformation methods may encompass NIST vetted cryptoraphic standards such as **format-preserving encryption (FPE)** via **FFE-1**, but can also be pseudonymous transformations of the data through other means, such as masking.

The secret engine currently supports `fpe`, `masking` and `tokenization` as data transformation types.

Prerequisites: 
* Vault Enterprise binary  
* Vault License

## Setup

In [ ]:
/bin/rm -rf $HOME/demos/vault_cluster/vrd/data/*

In [1]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [2]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [3]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.1+ent
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-b21c5852
Cluster ID      f49c7f18-415c-4dac-feaf-bf2adce232b5
HA Enabled      true
HA Cluster      https://0.0.0.0:8201
HA Mode         active
Last WAL        35


In [ ]:
which vault
vault --version

In [ ]:
echo $VAULT_ADDR

\
**EXECUTE the following command in SEPARATE window**

```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 evault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```
OR   
`evrd1`   

## (Optional)  Apply the license

In [5]:
curl \
    --insecure \
    --header "X-Vault-Token: $(vault print token)" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

In [ ]:
vault status

##### Notice Initialized (=true) and Sealed (=false) status 

# Demo

## Enable the Transform secrets engine

In [6]:
vault secrets enable transform

Success! Enabled the transform secrets engine at: transform/


## Roles, Transformations, Templates and Alphabets

The Transform secret engine contains several types resources that encapsulates different aspects of information required to perform transformation.  
**Roles** - holds the set of transformation allowed.  The role name is provided when performing encode and decode
  
**Transformations** - holds particular transformation.  It contains information about the type of transformation, template it should use for value detection, and other values such as tweak source or the masking character to use.
  
**Templates** - allow us to determine what and how to capture the value we want to transform
  
**Alphabets** - set of valid UTF-8 character contained within the input and transformed value on FPE transformations
<img src=images/vault-transform-3.png width="600">



## Use Case: Transform Credit Card
### Create a Role

In [ ]:
vault write transform/role/payments transformations=card-number

### Specify the transformation

In [ ]:
vault write transform/transformation/card-number \
  type=fpe \
  template=ccn \
  tweak_source=internal \
  allowed_roles=payments

### Optionally create a template (i.e. specify pattern for input)

In [ ]:
vault write transform/template/ccn \
  type=regex \
  pattern='(\d{4})-(\d{4})-(\d{4})-(\d{4})' \
  alphabet=numerics

### Optionally create an alphabet (a set of valid characters or numbers for the input)

In [ ]:
vault write transform/alphabet/numerics alphabet="0123456789"

### Transform it
After the secrets engine is configured and a user machine has a Vault token, it can use this secret engine to encode and decode input values

In [ ]:
vault write transform/encode/payments value=1111-2222-3333-4444

### Decode some input value using the /decode endpoint with a named role:

In [ ]:
vault write transform/decode/payments value=9093-6066-0313-5513

### Transform secrets
The Vault client applications must have the following policies to perform data encoding and ecoding using Tranform secrets engine enabled at `transform/`


> path "transform/encode/*" {  
> &nbsp; capabilities = [ "update" ]  
> }
> 
> path "transform/decode/*" {  
> &nbsp; capabilities = [ "update" ]  
> }


## Masking Credit card

In [ ]:
vault write transform/role/payments transformations=ccn-masking

In [ ]:
vault write transform/transformation/ccn-masking \
  type=masking \
  template="ccn" \
  masking_character=X \
  allowed_roles=payments

In [ ]:
vault write transform/template/ccn \
  type=regex \
  pattern='(\d{4})-(\d{4})-(\d{4})-\d\d\d\d' \
  alphabet=numerics

In [ ]:
vault write transform/encode/payments value=1111-2222-3333-4444

## Transform Singapore NRIC

In [7]:
vault write transform/role/payments transformations=card-number,ccn-masking,nric

Success! Data written to: transform/role/payments


### Create a template for NRIC (i.e. specify pattern)

In [8]:
vault write transform/template/nric \
  type=regex \
  pattern='[STFG]{1}(\d{7})[A-Z]{1}' \
  alphabet=builtin/numeric

Success! Data written to: transform/template/nric


In [9]:
vault write transform/transformation/nric \
  type=fpe \
  template=nric \
  tweak_source=internal \
  allowed_roles=payments

Success! Data written to: transform/transformation/nric


In [10]:
vault write transform/encode/payments value=T7780802G transformation=nric

Key              Value
---              -----
encoded_value    T9005337G


## Masking Singapore NRIC
Masking replaces matched characters on the input value with a desired character. 
Masking is unidirectional; therefor only encode is supported.


In [ ]:
vault write transform/role/payments transformations=card-number,ccn-masking,nric,mask-nric

In [ ]:
vault write transform/transformation/mask-nric \
        type=masking \
        template="nric" \
        masking_character=X \
        allowed_roles=payments

In [ ]:
vault write transform/template/nric \
  type=regex \
  pattern='[STFG]{1}(\d{7})[A-Z]{1}' \
  alphabet=builtin/numeric

In [ ]:
vault write transform/encode/payments value=S7180802G transformation="mask-nric"

### Batch Input Processing
You can send multiple secrets in a request payload as **batch_input** insetad of invoking API multiple times.

In [ ]:
#tee input-multiple.json <<EOF

cat <<EOF > /tmp/input-multiple.json
{
  "batch_input": [
    {
      "value": "1111-1111-1111-1111",
      "transformation": "card-number"
    },
    {
      "value": "S7780801G",
      "transformation": "nric"
    },
    {
      "value": "S7780801G",
      "transformation": "mask-nric"
    }
  ]
}
EOF


In [ ]:
export VAULT_TOKEN=root
curl -s --header "X-Vault-Token: $VAULT_TOKEN" \
     --request POST \
     --data @/tmp/input-multiple.json \
     $VAULT_ADDR/v1/transform/encode/payments | jq ".data"

In [ ]:
#tee payload-batch.json <<EOF

cat <<EOF > /tmp/input-multiple.json
{
  "batch_input": [
    { "value": "1111-1111-1111-1111", "transformation": "card-number" },
    { "value": "2222-2222-2222-2222", "transformation": "card-number" },
    { "value": "3333-3333-3333-3333", "transformation": "card-number" },
    { "value": "4444-4444-4444-4444", "transformation": "card-number" }
  ]
}
EOF


In [ ]:
curl -s --header "X-Vault-Token: $VAULT_TOKEN" \
     --request POST \
     --data @/tmp/input-multiple.json \
     $VAULT_ADDR/v1/transform/encode/payments | jq ".data"

### Next Steps
Before application can even request data transformation, it needs to authenticate with Vault.  THerefore an auth method (.e.g. AWS, K8s, AppRole) must be enabled and onfigured. In addition an appropriate policy must be created and attached to the client token

### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">